In [40]:
%load_ext autoreload
%autoreload 2

from InfluenceDiffusion import *
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# read email data from http://networkrepository.com/email-dnc.php
adj_list = pd.read_csv('data/email-dnc.edges',
                       skiprows=0,
                       delimiter=',',
                       names=['FROM', 'TO', 'TIME'])
# this is a multigraph
multigraph = nx.from_pandas_edgelist(adj_list[['FROM', 'TO']],
                                            source='FROM',
                                            target='TO',
                                            create_using=nx.MultiGraph)
# this is the directed graph, derived from the multigraph
directed_graph = nx.DiGraph()

for edge in set(multigraph.edges()):
    # get statistics
    u, v = edge[0], edge[1]
    num_edges_u_v = multigraph.number_of_edges(u, v)
    degree_u = multigraph.degree(u)
    degree_v = multigraph.degree(v)
    # construct graphs for influence models
    directed_graph.add_edge(u, v, c=num_edges_u_v, weight=num_edges_u_v / degree_v, du=degree_u, dv=degree_v)
    directed_graph.add_edge(v, u, c=num_edges_u_v, weight=num_edges_u_v / degree_u, du=degree_v, dv=degree_u)

In [3]:
result = begin_diffusion(initial_set={1},
                underlying_graph=directed_graph,
                max_iter=100,
                mode='linear threshold')
len(result)

MODE: linear threshold - EARLY TERMINATION


102

In [4]:
result = begin_diffusion(initial_set={1},
                underlying_graph=directed_graph,
                max_iter=100,
                mode='independent cascade',
                         p_cascade=0.01)
len(result)

MODE: independent cascade - EARLY TERMINATION


177

In [5]:

result = begin_diffusion(initial_set={1},
                underlying_graph=directed_graph,
                max_iter=100,
                mode='weighted cascade')
len(result)

MODE: weighted cascade - EARLY TERMINATION


90

In [55]:
greedy_initial_set = hill_climbing_greedy(directed_graph, k=5, max_iter=300, p_cascade=0.01, mode='linear threshold')